<h1>Fase 1: Exploración y Limpieza</h1>

<h2> Exploración Inicial:</h2>

- [ ] Realiza una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes.
- [ ] Utiliza funciones de Pandas para obtener información sobre la estructura de los datos, la presencia de valores nulos y estadísticas básicas de las columnas involucradas.
- [ ] Une los dos conjuntos de datos de la forma más eficiente.

In [12]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Para imputación de nulos usando métodos estadísticos avanzados
from sklearn.impute import SimpleImputer 
from sklearn.experimental import enable_iterative_imputer # Necesario para IterativeImputer
from sklearn.impute import IterativeImputer 
from sklearn.impute import KNNImputer 

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [13]:
# Cargamos el primer dataset y vemos las 5 primeras entradas para asegurarnos de que lo hemos hecho bien
clh_df = pd.read_csv("Customer Loyalty History.csv")
clh_df.head()

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [14]:
# Cargamos el segundo dataset y vemos las 5 primeras entradas para asegurarnos de que lo hemos hecho bien

cfa_df = pd.read_csv("Customer Flight Activity.csv")
cfa_df.head()

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [24]:
# Aquí hacemos una primera exploración superficial:

print("'Customer Loyalty History' dataset")
print()
print(clh_df.info()) # vemos la información general
print()
print(clh_df.isnull().sum()) # vemos la suma de valores nulos que tenemos
print()
print()
print()
print("'Customer Flight activity' dataset")
print()
print(cfa_df.info())
print()
print(cfa_df.isnull().sum())

'Customer Loyalty History' dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Mont

In [30]:
# Vemos la cantidad de duplicados que tenemos:

print(cfa_df.duplicated().sum())
print(clh_df.duplicated().sum())

1864
0


En el caso de la información sobre duplicados, vemos que para Customer Loyalty History no tenemos ningún duplicado. Para Customer Flight Activity, sin embargo, tenemos bastantes. 

In [32]:
for col in clh_df.select_dtypes(include='O').columns: # select_dtypes selecciona solo columnas de tipo objeto y .collumns devuelve la lista de nombres de esas columnas
    print(f"Las categorías que tenemos para la columna {col} son:")
    display(clh_df[col].value_counts().reset_index()) # cuenta cuántas veces aparece cada valor en la columna, reset_index()
                                                      # convierte el resultado en DF y display muestra el DF más legible que un print
    print('_' * 100) # esto imprime una línea de guiones para separar visualmente cada columna

Las categorías que tenemos para la columna Country son:


,Country,count
0,Canada,16737


____________________________________________________________________________________________________
Las categorías que tenemos para la columna Province son:


,Province,count
0,Ontario,5404
1,British Columbia,4409
2,Quebec,3300
3,Alberta,969
4,Manitoba,658
5,New Brunswick,636
6,Nova Scotia,518
7,Saskatchewan,409
8,Newfoundland,258
9,Yukon,110


____________________________________________________________________________________________________
Las categorías que tenemos para la columna City son:


...

____________________________________________________________________________________________________
Las categorías que tenemos para la columna Loyalty Card son:


,Loyalty Card,count
0,Star,7637
1,Nova,5671
2,Aurora,3429


____________________________________________________________________________________________________
Las categorías que tenemos para la columna Enrollment Type son:


,Enrollment Type,count
0,Standard,15766
1,2018 Promotion,971


____________________________________________________________________________________________________


In [31]:
# Aquí vemos la distribución de algunas variables categóricas, para asegurarnos de que no hay valores atípicos

# Con este bucle, recorremos el conjunto de variables categóricas y vemos la proporción de las 10 categorías más comunes
for i in ["Country", "Gender", "Education", "Marital Status", "Loyalty Card", "Enrollment Type"]:
    print (f"{i}:", clh_df[i].value_counts(normalize = True).head(20)) # con normalize se nos da la proporción en lugar del número absoluto

Country: Country
Canada    1.0
Name: proportion, dtype: float64
Gender: Gender
Female    0.50248
Male      0.49752
Name: proportion, dtype: float64
Education: Education
Bachelor                0.625859
College                 0.253211
High School or Below    0.046723
Doctor                  0.043855
Master                  0.030352
Name: proportion, dtype: float64
Marital Status: Marital Status
Married     0.581645
Single      0.267909
Divorced    0.150445
Name: proportion, dtype: float64
Loyalty Card: Loyalty Card
Star      0.456294
Nova      0.338830
Aurora    0.204875
Name: proportion, dtype: float64
Enrollment Type: Enrollment Type
Standard          0.941985
2018 Promotion    0.058015
Name: proportion, dtype: float64


Podemos ir adelantando lo siguiente:
- Tenemos un único mercado
- La distribución en función del género está equilibrada (aunque probablemente debería añadirse, como mínimo, una tercera opción)
- La mayoría de los clientes tiene estudios superiores
- La mayoría de los clientes tiene pareja
- La mayoría de los clientes está en el nivel más bajo de la tarjeta de fidelización (45.6%)
- El programa no depende de promociones para atraer clientes, puesto que la gran mayoría (más del 90%) no entra por promoción

In [28]:
# Aquí, con describe, vemos datos estadísticos básicos de las variables numéricas

print(clh_df[["Salary", "CLV"]].describe())

              Salary           CLV
count   12499.000000  16737.000000
mean    79245.609409   7988.896536
std     35008.297285   6860.982280
min    -58486.000000   1898.010000
25%     59246.500000   3980.840000
50%     73455.000000   5780.180000
75%     88517.500000   8940.580000
max    407228.000000  83325.380000


De estos datos podemos sacar estas conclusiones de cara a futuras acciones de limpiado:
- Hay datos negativos en el salario que no tienen sentido
- Hay algunos outliers con salarios muy altos

In [29]:
print(cfa_df[["Distance", "Points Accumulated", "Points Redeemed", "Flights Booked"]].describe())

            Distance  Points Accumulated  Points Redeemed  Flights Booked
count  405624.000000       405624.000000    405624.000000   405624.000000
mean     1208.880059          123.692721        30.696872        4.115052
std      1433.155320          146.599831       125.486049        5.225518
min         0.000000            0.000000         0.000000        0.000000
25%         0.000000            0.000000         0.000000        0.000000
50%       488.000000           50.000000         0.000000        1.000000
75%      2336.000000          239.000000         0.000000        8.000000
max      6293.000000          676.500000       876.000000       21.000000


De estos datos podemos sacar estas conclusiones de cara a futuras acciones de limpiado:
- En el caso de los puntos acumulados, se concentran en una minoría